# 1. Instalar y cargar librería

In [1]:
# instalar previamente
#!pip install plotly
#!pip install cufflinks

In [2]:
# Importar la biblioteca pandas para manipulación y análisis de datos
import pandas as pd
import cufflinks as cf
from IPython.display import display,HTML
import os
import numpy as np
import time
inicio_todo_el_proceso = time.time()
cf.set_config_file(sharing = 'public',theme = 'ggplot', offline = True)

# 2. Cargar base de datos y definir, campos a analizar y método a utilizar

In [3]:
# Definir una función para cargar la base de datos desde un archivo CSV
def cargar_base_datos(url):
    try:
        # Leer el archivo CSV, saltar líneas incorrectas y guardar el contenido en un DataFrame
        df = pd.read_csv(url, on_bad_lines='skip',low_memory=False)
        return df
    except FileNotFoundError:
        # Manejar el caso en que el archivo no se encuentre y mostrar un mensaje de error
        print(f"Archivo no encontrado: {url}")
        return None

# Definir una función para limpiar el título de los registros en un DataFrame
def limpiar_titulo(df):
    # Separar el título por el caracter '[' y conservar la primera parte
    df['Title'] = df['Title'].str.split('[').str[0]
    return df

# Diccionarios para mapear opciones de configuración

#Campo de la base de datos a analizar
dic_campo_a_analizar = {
    1: 'key_words',
    2: 'title',
    3: 'abstract',
    4: 'title_key',
    5: 'title_abs',
    6: 'key_abs',
    7: 'title_abs_key',
}

#Método elegido para la limpieza de palabras
dic_metodo_a_utilizar = {1: 'lemmatize', 2: 'stemming'}

#1 si muestra el grafo como conteo de nodos y aristas 
#2 si muestra el grafo como porxentaje de la cantidad total de artículos
dic_modo_grafo = {1: '_ABS_', 2: '_POND_'} 

# Variables de configuración
metodo = 1
campo_a_analizar = 7
template_elegido = 'seaborn'
modo_grafo = 2

# Definición de abreviaturas y nombres para la base de datos
abrev = 'V4_algoritmo_2'
nombre_largo = '37 revistas de Educación  Matemática '
fuente_articulos = 'Scopus'


# Ruta del archivo CSV
url1 = '/Users/jorgegaona/Dropbox/Artículos JG/(2023) Análisi ERC 38 revistas educación matemática SCOPUS y WOS/bbdd_unida_completa_37_revistas_V_2023_03_25.csv'
url = url1

# Cargar la base de datos y asignarla a la variable dfA
dfA = cargar_base_datos(url)

# Verificar si la base de datos se cargó correctamente
if dfA is not None:
    # Imprimir el tamaño de la base de datos
    print('Tamaño base de datos: ', len(dfA))
    # Limpiar los títulos en la base de datos
    dfA = limpiar_titulo(dfA)
    # Imprimir nuevamente el tamaño de la base de datos
    print(len(dfA))
    # Mostrar el primer registro del DataFrame
    dfA.head(1)
else:
    # Imprimir un mensaje de error en caso de que la base de datos no se haya cargado
    print("No se pudo cargar la base de datos.")

dfA.head(1)


Tamaño base de datos:  23094
23094


,Unnamed: 0,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID
0,0,"Oldknow A., White J., Cross J.",6507266545;55568510911;36510330000;,Micromaths,1985,Teaching Mathematics and its Applications,4.0,1,NaN,17,...,NaN,NaN,NaN,English,Teach. Math. Appl.,Article,Final,NaN,Scopus,2-s2.0-77957219102


# 2. Para unir varios data frame 

# 3 Creación de carpeta donde se guardará la información

In [4]:
import os
from datetime import datetime

# Obtener el momento actual como un objeto datetime
now = datetime.now()

# Crear un directorio con el nombre almacenado en la variable 'abrev', si no existe
if not os.path.exists(abrev):
    os.mkdir(abrev)

# Imprimir el momento actual en la consola
print(now)


2023-04-27 23:54:33.296373


# 4 Cargar archivos para el informe generado por Latex

In [5]:
from urllib import request

# Establecer la ruta del directorio donde se guardarán los archivos
ruta_archivo = abrev + '/'

# Definir las URL de los archivos a descargar
remote_url1 = "https://www.dropbox.com/s/alwmw7pt709tats/apalike-es.bst?dl=1"
remote_url2 = "https://www.dropbox.com/s/itwnlprry4guwfj/referencias.bib?dl=1"

# Definir los nombres de los archivos locales a guardar
local_file1 = 'apalike-es.bst'
local_file2 = "referencias.bib"

# Descargar y guardar el primer archivo de la URL remota en la ruta local
request.urlretrieve(remote_url1, ruta_archivo + local_file1)

# Descargar y guardar el segundo archivo de la URL remota en la ruta local
request.urlretrieve(remote_url2, ruta_archivo + local_file2)


('V4_algoritmo_2/referencias.bib', <http.client.HTTPMessage at 0x7fa00c5c39a0>)

# 5 Creación de funciones para escribir entorno latex

In [6]:
import os

# Crear un nuevo archivo .tex y escribir el inicio del documento
def entorno_abrir_tex(ruta_archivo, nombre_archivo):
    with open(ruta_archivo + nombre_archivo + ".tex", "w") as file:
        file.write("% Comienza el texto" + os.linesep)
        file.write("\n" + os.linesep)

# Crear un nuevo archivo .bib y escribir el inicio del documento
def entorno_abrir_bib(ruta_archivo, nombre_archivo):
    with open(ruta_archivo + nombre_archivo + ".bib", "w") as file:
        file.write("% Comienza el texto" + os.linesep)
        file.write("\n" + os.linesep)

# Abrir un archivo .tex existente y agregar el entorno del documento
def entorno_abrir_documento_tex(ruta_archivo, nombre_archivo):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file.write('\\' + "begin{document}" + os.linesep)
        file.write("\n" + os.linesep)

# Cerrar el entorno del documento en un archivo .tex existente
def entorno_cerrar_documento_tex(ruta_archivo, nombre_archivo):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file.write('\\' + 'end{document}' + os.linesep)

# Agregar un entorno de figura en un archivo .tex existente
def entorno_figura_tex(ruta_archivo, nombre_archivo, nombre_archivo_figura, leyenda):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file.write('\\' + "begin{figure}[h!]" + os.linesep)
        file.write('\\' + "centering" + os.linesep)
        file.write("\includegraphics[width=0.7" + '\\' + "textwidth]{" + nombre_archivo_figura + ".pdf}" + os.linesep)
        file.write('\\' + "caption{" + leyenda + '}' + os.linesep)
        file.write('\\' + 'label{fig:' + nombre_archivo_figura + '}' + os.linesep)
        file.write('\\' + "end{figure}" + os.linesep)
        file.write("\n" + os.linesep)

# Agregar una sección en un archivo .tex existente
def entorno_section_tex(ruta_archivo, nombre_archivo, nombre_sección):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file.write('\\' + 'section{' + nombre_sección + '}' + os.linesep)
        file.write("\n" + os.linesep)

# Agregar una subsección en un archivo .tex existente
def entorno_sub_section_tex(ruta_archivo, nombre_archivo, nombre_sub_sección):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file.write('\\' + 'subsection{' + nombre_sub_sección + '}' + os.linesep)
        file.write("\n" + os.linesep)

# Agregar una subsubsección en un archivo .tex existente
def entorno_sub_sub_section_tex(ruta_archivo, nombre_archivo, nombre_sub_sub_sección):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file.write('\\' + 'subsubsection{' + nombre_sub_sub_sección + '}' + os.linesep)
        file.write("\n" + os.linesep)

# Agregar un entorno itemize en un archivo .tex existente
def entorno_itemize_tex(ruta_archivo, nombre_archivo, lista_para_itemize):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file.write('\\' + 'begin{itemize}' + os.linesep)
        for x in lista_para_itemize:
            file.write('\\' + 'item ' + x + os.linesep)
        file.write('\\' + 'end{itemize}' + os.linesep)
        file.write("\n" + os.linesep)

# Agregar un entorno enumerate en un archivo .tex existente
def entorno_enumerate_tex(ruta_archivo, nombre_archivo, lista_para_enumerate):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file.write('\\' + 'begin{enumerate}' + os.linesep)
        for x in lista_para_enumerate:
            file.write('\\' + 'item ' + x + os.linesep)
        file.write('\\' + 'end{enumerate}' + os.linesep)
        file.write("\n" + os.linesep)

# Agregar una nueva página en un archivo .tex existente
def entorno_nueva_pagina_tex(ruta_archivo, nombre_archivo):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file.write('\\' + 'newpage' + os.linesep)
        file.write("\n" + os.linesep)

# Inicializar el documento .tex con los paquetes necesarios y el título
def entorno_inicio_documento_tex(ruta_archivo, nombre_archivo, título):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file.write('\\documentclass[12pt,a4paper]{article}\n')
        file.write('\\usepackage[utf8]{inputenc}\n')
        file.write('\\usepackage[spanish, es-tabla]{babel}\n')
        file.write('\\usepackage[hidelinks,colorlinks=true,citecolor=blue, backref=page]{hyperref}\n')
        file.write('\\usepackage{amsmath}\n')
        file.write('\\usepackage{amsfonts}\n')
        file.write('\\usepackage{multirow}\n')
        file.write('\\usepackage{booktabs}\n')
        file.write('\\usepackage{apacite}\n')
        file.write('\\usepackage{multirow}\n')
        file.write('\\usepackage{natbib}\n')
        file.write('\\usepackage{amssymb}\n')
        file.write('\\usepackage{graphicx}\n')
        file.write('\\usepackage[width=18.00cm, height=27.00cm]{geometry}\n')
        file.write('\\title{' + título + '}\n')
        file.write('\\begin{document}\n')
        file.write('\\maketitle \n')

# Agregar un párrafo en un archivo .tex existente
def parrafo_tex(ruta_archivo, nombre_archivo, parrafo):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file.write(parrafo + os.linesep)
        file.write("\n" + os.linesep)

# Agregar una tabla:
def crear_tabla_latex(latex_table, caption, label):
    # Agregar el código LaTeX para el entorno table
    table = '\\begin{table}[h!]\n'
    table += '\\centering\n'
    table += '\\caption{' + caption + '}\n'
    table += '\\label{' + label + '}\n'
    table += latex_table
    table += '\\end{table}'
    
    # Devolver el código LaTeX del entorno table
    return table

    

# 6 Primera parte del documento latex con el preámbulo

In [7]:
# Definir la ruta del archivo y el nombre del archivo .tex
ruta_archivo = abrev + '/'
nombre_archivo_tex = '01_informe_' + abrev + str(now.year) + '_' + str(now.month) + '_' + str(now.day) + '_' + str(now.hour) + '_' + str(now.minute)

# Crear y abrir el archivo .tex
entorno_abrir_tex(ruta_archivo, nombre_archivo_tex)

# Definir el título del documento
título = 'Espacio Rizomático Conceptual de \\textbf{' + nombre_largo + '}'

# Inicializar el documento .tex con los paquetes necesarios y el título
entorno_inicio_documento_tex(ruta_archivo, nombre_archivo_tex, título)

# Agregar una sección 'Introducción' en el archivo .tex
entorno_section_tex(ruta_archivo, nombre_archivo_tex, 'Introducción')

# Crear el primer párrafo de la sección 'Introducción'
lista_párrafo1 = ['Este documento muestra el análisis en base al trabajo de \\cite{Gaona2023}. Se realizó a partir de una base de datos con ',
                  str(len(dfA)),
                  ' artículos. Estos artículos pertenecen a ',
                  str(len(dfA['Source title'].unique()))
                  ]

# Decidir si el texto debe ser singular o plural en función de la cantidad de fuentes documentales
if len(dfA['Source title'].unique()) == 1:
    lista_párrafo1_1 = ' fuente documental.'
else:
    lista_párrafo1_1 = ' fuentes documentales diferentes.'

# Añadir la parte correspondiente del texto al primer párrafo
lista_párrafo1.append(lista_párrafo1_1)
lista_párrafo1 = lista_párrafo1 + [' Los artículos fueron publicados ']

# Decidir si el texto debe incluir un solo año o un rango de años en función de los datos
if len(dfA['Year'].unique()) == 1:
    lista_párrafo1_2 = ' el año ' + str(dfA['Year'].unique()) + '.'
else:
    lista_párrafo1_2 = ' entre los años ' + str(dfA['Year'].min()) + ' y ' + str(dfA['Year'].max()) + '.'

# Añadir la parte correspondiente del texto al primer párrafo
lista_párrafo1.append(lista_párrafo1_2)
lista_párrafo1 = lista_párrafo1 + [' La información se extrajo de', fuente_articulos + '.']

# Unir la lista en una cadena de texto para formar el primer párrafo
párrafo1 = ' '.join(lista_párrafo1)
print(párrafo1)

# Agregar el primer párrafo al archivo .tex
parrafo_tex(ruta_archivo, nombre_archivo_tex, párrafo1)


Este documento muestra el análisis en base al trabajo de \cite{Gaona2023}. Se realizó a partir de una base de datos con  23094  artículos. Estos artículos pertenecen a  43  fuentes documentales diferentes.  Los artículos fueron publicados   entre los años 1968 y 2023.  La información se extrajo de Scopus.


# 7 Documento LaTex con el tabla de cantidad de artículos por años y citas

In [8]:
# Crear una copia del DataFrame original
dfB = dfA.copy()

# Renombrar la columna 'Year' a 'Año'
dfB.rename(columns={'Year': 'Año'}, inplace=True)

# Agrupar el DataFrame por año y calcular el tamaño y la suma de la columna 'Cited by'
dfA_summary = dfB.groupby('Año')['Cited by'].agg(['size', 'sum'])

# Renombrar las columnas del DataFrame agrupado
dfA_summary.rename(columns={'size': 'Nº artículos', 'sum': 'Nº de citas'}, inplace=True)

# Restablecer el índice del DataFrame
dfA_summary.reset_index(inplace=False)

# Ordenar el DataFrame por año de forma ascendente
dfA_summary.sort_values(by='Año', ascending=True, inplace=True)

# Determinar la cantidad de años a mostrar en el resumen
años_a_mostrar = 15
if dfA['Year'].max() - dfA['Year'].min() + 1 > años_a_mostrar:
    años_a_mostrar = años_a_mostrar
else:
    años_a_mostrar = dfA['Year'].max() - dfA['Year'].min() + 1

# Seleccionar los últimos n años para mostrar en el resumen
dfA_summary = dfA_summary.tail(años_a_mostrar)

# Convertir la columna 'Nº de citas' a enteros
dfA_summary['Nº de citas'] = dfA_summary['Nº de citas'].apply(int)

# Convertir el DataFrame en una tabla LaTeX
latex_table = dfA_summary.to_latex()


# Crear la tabla LaTeX con un título y una etiqueta
caption = 'Nº de publicaciones y cantidad de citas de los últimos ' + str(años_a_mostrar) + ' años.'
label = 'tab: nº publicaciones y citas'
latex_table = crear_tabla_latex(latex_table, caption, label)

# Agregar la tabla LaTeX al archivo informe.tex
crear_tabla_latex(ruta_archivo, nombre_archivo_tex, latex_table)

# Imprimir el resumen
print(dfA_summary)
print(caption)


      Nº artículos  Nº de citas
Año                            
2009           558         9351
2010           599         8315
2011           701         8505
2012           749         8798
2013           862        10766
2014           894         9335
2015           915        10794
2016           930         9650
2017          1285        12277
2018          1225         9143
2019          1221         6095
2020          1227         5658
2021          1355         3152
2022          1671         1291
2023            12           20
Nº de publicaciones y cantidad de citas de los últimos 15 años.


# 8 Documento LaTex con el gráfico de cantidad de artículos por años y citas

In [9]:
import plotly.express as px


######## Gráfico de nº de artículos ############

# Agrupar el DataFrame por año y contar el número de EID en cada grupo
dfA_plot = dfA.groupby('Year')['EID'].count().reset_index()

# Crear un gráfico de barras usando Plotly Express
fig = px.bar(dfA_plot, x='Year', y='EID', template=template_elegido)

# Personalizar el título, los ejes x e y del gráfico
fig.update_layout(title='Nº de artículos en los últimos ' + str(años_a_mostrar) + ' años')
fig.update_xaxes(range=[dfA['Year'].max() - años_a_mostrar - 0.5, dfA['Year'].max() + 0.5-1])
fig.update_yaxes(title='Nº de artículos')
fig.update_xaxes(title='Año')

# Mostrar el gráfico
fig.show()

# Crear un nombre de archivo para guardar el gráfico
nombre_archivo = abrev + '_' + "_n_" + str(len(dfA)) + 'articulos_ultimos_' + str(años_a_mostrar) + '_anos_' + str(
    now.year) + '_' + str(now.month) + '_' + str(now.day) + '_' + str(now.hour) + '_' + str(now.minute)

# Crear un párrafo con información sobre el gráfico
lista_párrafo2 = ['De los ', str(len(dfA)), ' documentos analizados en este informe, ',
                  str(len(dfA[dfA['Year'] >= dfA['Year'].max() - años_a_mostrar])),
                  ' fueron realizados en los últimos ', str(años_a_mostrar), ' años, es decir, el',
                  str(round(100 * len(dfA[dfA['Year'] >= dfA['Year'].max() - años_a_mostrar]) / len(dfA), 1)),
                  '\\% del total. Su distribución se muestra en la figura \\ref{fig:' + nombre_archivo + '}.']

párrafo2 = ' '.join(lista_párrafo2)
print(párrafo2)

# Agregar el párrafo al archivo informe.tex
parrafo_tex(ruta_archivo, nombre_archivo_tex, párrafo2)

# Guardar el gráfico como un archivo HTML y PDF
fig.write_html(abrev + '/' + nombre_archivo + ".html")
fig.write_image(abrev + '/' + nombre_archivo + ".pdf", width=600, height=300)

# Agregar la figura al archivo informe.tex
nombre_archivo_figura = nombre_archivo
leyenda = 'Artículos publicados en ' + fuente_articulos + ' de ' + nombre_largo + '. Fuente: ' + fuente_articulos + '.'
entorno_figura_tex(ruta_archivo, nombre_archivo_tex, nombre_archivo_figura, leyenda)







######## Gráfico de citas############

# Agrupar los datos por año y calcular la suma de las citas para cada año
dfA_plot = dfA.groupby('Year')['Cited by'].sum().reset_index()

# Crear un gráfico de barras usando Plotly Express
fig = px.bar(dfA_plot, x='Year', y='Cited by', template=template_elegido)

# Actualizar el título del gráfico
fig.update_layout(title='Nº de citas en los últimos ' + str(años_a_mostrar) + ' años')

# Ajustar el rango del eje x y agregar un margen adicional para evitar recortes en las barras extremas
fig.update_xaxes(range=[dfA['Year'].max() - años_a_mostrar - 0.5, dfA['Year'].max() + 0.5])

# Actualizar el título del eje y
fig.update_yaxes(title='Nº de citas')

# Actualizar el título del eje x
fig.update_xaxes(title='Año')

# Mostrar el gráfico
fig.show()

# Crear el nombre del archivo
nombre_archivo = abrev + '_' + "_n_" + str(len(dfA)) + 'citas_ultimos_' + str(años_a_mostrar) + '_anos_' + str(now.year) + '_' + str(now.month) + '_' + str(now.day) + '_' + str(now.hour) + '_' + str(now.minute)

# Crear el párrafo para el informe
lista_párrafo3 = ['Los documentos tienen en total  ', str(dfA['Cited by'].sum().astype(int)), ' citas. De las cuales,  ',
                  str((dfA[dfA['Year'] >= dfA['Year'].max() - años_a_mostrar]['Cited by'].sum()).astype(int)),
                  ' fueron realizados en los últimos ', str(años_a_mostrar), ' años.',
                  'Su distribución se muestra en la figura \\ref{fig:' + nombre_archivo + '}.']

párrafo3 = ' '.join(lista_párrafo3)
print(párrafo3)

# Agregar el párrafo al archivo LaTeX
parrafo_tex(ruta_archivo, nombre_archivo_tex, párrafo3)

# Guardar el gráfico como archivo HTML y PDF
fig.write_html(abrev + '/' + nombre_archivo + ".html")
fig.write_image(abrev + '/' + nombre_archivo + ".pdf", width=600, height=300)

# Crear la leyenda de la figura
nombre_archivo_figura = nombre_archivo
leyenda = 'Citas de artículos publicados en ' + fuente_articulos + ' de ' + nombre_largo + '. Fuente: ' + fuente_articulos + '.'

# Agregar la figura y su leyenda al archivo LaTeX
entorno_figura_tex(ruta_archivo, nombre_archivo_tex, nombre_archivo_figura, leyenda)

# Imprimir la leyenda
print(leyenda)



De los  23094  documentos analizados en este informe,  14775  fueron realizados en los últimos  15  años, es decir, el 64.0 \% del total. Su distribución se muestra en la figura \ref{fig:V4_algoritmo_2__n_23094articulos_ultimos_15_anos_2023_4_27_23_54}.


Los documentos tienen en total   209053  citas. De las cuales,   122903  fueron realizados en los últimos  15  años. Su distribución se muestra en la figura \ref{fig:V4_algoritmo_2__n_23094citas_ultimos_15_anos_2023_4_27_23_54}.
Citas de artículos publicados en Scopus de 37 revistas de Educación  Matemática . Fuente: Scopus.


# 9 Conteo de todos los campos antes de tratar la información

In [10]:
import plotly.express as px

# Filtrar las filas con palabras clave de autor no nulas
dfK = dfA[dfA['Author Keywords'].notnull()]

# Crear una lista de palabras clave separadas por punto y coma
dfK['key_words'] = dfK['Author Keywords'].apply(lambda x: x.split(';'))

# Eliminar espacios en blanco de las palabras clave
dfK['key_words'] = dfK['key_words'].apply(lambda x: [word.strip() for word in x])

# Convertir todas las palabras clave a minúsculas
def minus(lista):
    return list(map(lambda x: x.lower(), lista))

dfK['key_words'] = dfK['key_words'].transform(minus)

# Crear una lista de palabras clave ordenadas alfabéticamente
key_word_list0 = sorted(dfK['key_words'].tolist())

# Contar la frecuencia de cada palabra clave
from collections import Counter

key_word_count0 = dict(Counter(i for sub in key_word_list0 for i in set(sub)))
key_word_count0.pop('nan', None)

# Ordenar las palabras clave por frecuencia
key_word_list0_ordenado = sorted(key_word_count0.items(), key=lambda x: x[1], reverse=True)

# Seleccionar las n palabras clave más frecuentes
n = 35
key_word_list0_ordenado_elegidos = key_word_list0_ordenado[:n]

##### Gráfico de palabras clave más frecuentes ###


# Calcular porcentajes
porcentajes = [round((frecuencia / len(dfA)) * 100, 1) for palabra, frecuencia in key_word_list0_ordenado_elegidos]


bar_texts = [f"{frecuencia} ({porcentaje}%)" for frecuencia, porcentaje in zip([key_word_list0_ordenado_elegidos[j][1] for j in range(0, n - 1)], porcentajes)]



#Crear títulos y gráfico
title = str(n) + ' palabras clave más frecuentes de ' + nombre_largo

# Crear el gráfico de barras
fig = px.bar(template=template_elegido, orientation='h',
             y=[key_word_list0_ordenado_elegidos[j][0] for j in range(0, n - 1)],
             x=[key_word_list0_ordenado_elegidos[j][1] for j in range(0, n - 1)], text=bar_texts,
             )

    

# Actualizar el título, los títulos de los ejes y el ángulo de las etiquetas del eje x
fig.update_layout(
    title=title,
    yaxis_title="palabras clave",
    xaxis_title="frecuencia",
    xaxis=dict(range=[0, max([key_word_list0_ordenado_elegidos[j][1] for j in range(0, n - 1)]) * 1.25]))  # Aumentar el rango del eje x en un 15%)
fig.update_xaxes(tickangle=35)
fig.update_layout(font_size=8)
# Actualizar el tamaño de la fuente del texto de las barras
fig.update_traces(textposition='outside', textfont=dict(size=14))  # Ajustar el tamaño y la posición del texto

# Mostrar el gráfico
fig.show()

# Crear el nombre del archivo
nombre_archivo = abrev + "_" + str(n) + '_palabras_claves_mas_frecuentes_' + str(now.year) + '_' + str(now.month) + '_' + str(now.day) + '_' + str(now.hour) + '_' + str(now.minute)

# Guardar el gráfico como archivo HTML y PDF
fig.write_html(abrev + '/' + nombre_archivo + ".html")
fig.write_image(abrev + '/' + nombre_archivo + ".pdf", width=700, height=800)
nombre_archivo_figura = nombre_archivo

# Agregar una sección al archivo LaTeX
entorno_section_tex(ruta_archivo, nombre_archivo_tex, 'Palabras clave de la base de datos')

# Crear el párrafo para el informe
lista_párrafo4 = ['En la figura \\ref{fig:' + nombre_archivo + '}',
                  'se observan las ', str(n), 'palabras clave con mayor frecuencia en la',
                  'base de datos analizada. La palabra que más aparece es <<', key_word_list0_ordenado_elegidos[0][0], '>>,',
                  'la cual, se repite ', str(key_word_list0_ordenado_elegidos[0][1]), 'veces,',
                  'es decir, en ', str(round(100 * key_word_list0_ordenado_elegidos[0][1] / len(dfA), 1)),
                  '\% de los documentos.']

párrafo4 = ' '.join(lista_párrafo4)

# Añadir el párrafo al archivo LaTeX
parrafo_tex(ruta_archivo, nombre_archivo_tex, párrafo4)

# Crear la leyenda para el gráfico
leyenda = 'Palabras clave proporcionadas por los autores en cada uno de los '+str(len(dfA))+' artículos analizados. Fuente: ' + fuente_articulos + '. Solo se muestran las palabras clave más frecuentes porque en el título y abstract el proceso implica transformarlos.'

# Añadir el entorno de figura al archivo LaTeX
entorno_figura_tex(ruta_archivo, nombre_archivo_tex, nombre_archivo_figura, leyenda)
print(leyenda)

# Imprimir las palabras clave más frecuentes

# Crear una lista de palabras clave y sus frecuencias
lista_palabras_clave = [key_word_list0_ordenado_elegidos[j][0] + ': ' + str(key_word_list0_ordenado_elegidos[j][1]) for j in range(0, n - 1)]



Palabras clave proporcionadas por los autores en cada uno de los 23094 artículos analizados. Fuente: Scopus. Solo se muestran las palabras clave más frecuentes porque en el título y abstract el proceso implica transformarlos.


# 10 Importar librerías sobre tratamiento del lenguaje natural

In [11]:
# Importar la librería nltk
import nltk

# Descargar el paquete "popular" de nltk (si es necesario), esto incluye recursos como corpus y modelos para el procesamiento del lenguaje natural
# nltk.download('popular')

# Importar módulos adicionales
import re
import string

# Inicializar el lematizador de WordNet y el stemmer de Porter
wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()

# Obtener la lista de palabras vacías (stopwords) en inglés de nltk
stopwords = nltk.corpus.stopwords.words('english')

# Agregar palabras adicionales a la lista de stopwords
stopwords.append('The')
stopwords.append('use')


# 11  Limpieza de conectores e ilativos

In [12]:
# Función para limpiar el texto, eliminando puntuación y palabras vacías (stopwords)
def clean_text(txt):
    # Eliminar puntuación
    txt = "".join([c for c in txt if c not in string.punctuation])
    # Tokenizar el texto utilizando expresiones regulares
    tokens = re.split('\W+', txt)
    # Eliminar palabras vacías (stopwords)
    txt = [word for word in tokens if word not in stopwords]
    return txt

# Lista de palabras adicionales a eliminar
lista_eliminar = ['al', 'ano', 'año', 'com', 'commun', 'como', 'con', 'da', 'de', 'del', 'desd', 'do', 'e', 'el', 'em', 'en','et', 'entr', 'la','las', 'lo','los', 'na', 'para', 'por', 'que','sobre', 'sobr', 'su', 'sua', 'toward', 'two','um', 'uma', 'un', 'una', 'use', 'uso']

# Función para eliminar palabras específicas de una lista
def eliminar_palabras(lista):
    for x in range(0, len(lista_eliminar)):
        if lista_eliminar[x] in lista:
            lista.remove(lista_eliminar[x])            
    return lista

# Aplicar la limpieza en la columna 'Author Keywords' si existe
if 'Author Keywords' in dfA.columns:
    dfA['Author Keywords'] = dfA['Author Keywords'].astype(str)
    dfA['Author Keywords'] = dfA['Author Keywords'].str.lower()
    dfA['key_words_non_spot'] = dfA['Author Keywords'].apply(lambda x: clean_text(x))

# Aplicar la limpieza en la columna 'Title' si existe
if 'Title' in dfA.columns:
    dfA['Title'] = dfA['Title'].astype(str)
    dfA['Title'] = dfA['Title'].str.lower()
    dfA['title_non_spot'] = dfA['Title'].apply(lambda x: clean_text(x))

# Aplicar la limpieza en la columna 'Abstract' si existe
if 'Abstract' in dfA.columns:
    dfA['Abstract'] = dfA['Abstract'].astype(str)
    dfA['Abstract'] = dfA['Abstract'].str.lower()
    dfA['abstract_non_spot'] = dfA['Abstract'].apply(lambda x: clean_text(x))


# 12 Aplicar método lemmatize o stemming

In [13]:
# Función que lematiza las palabras en un texto tokenizado
def lemmatization(token_txt):
    text = [wn.lemmatize(word) for word in token_txt]
    return text

# Función que aplica stemming en un texto tokenizado
def stemming(token_txt):
    text = [ps.stem(word) for word in token_txt]
    return text

# Aplicar lematización o stemming en función del valor de 'metodo'
if metodo == 1:
    met = 'lemmatize'
    # Aplicar lematización a las columnas 'key_words_non_spot', 'title_non_spot' y 'abstract_non_spot'
    processing_function = lemmatization
else:
    met = 'stemming'
    # Aplicar stemming a las columnas 'key_words_non_spot', 'title_non_spot' y 'abstract_non_spot'
    processing_function = stemming

# Crear columnas 'key_words_metodo', 'title_metodo' y 'abstract_metodo'
for column in ['key_words_non_spot', 'title_non_spot', 'abstract_non_spot']:
    dfA[f'{column[:-9]}_metodo'] = dfA[column].apply(processing_function)
    dfA[f'{column[:-9]}_metodo'] = dfA[f'{column[:-9]}_metodo'].apply(lambda x: list(set(x)))
    dfA[f'{column[:-9]}_metodo'] = dfA[f'{column[:-9]}_metodo'].apply(eliminar_palabras)

# Funciones para combinar palabras de diferentes columnas
def combinar_title_key(row):
    return row['title_metodo'] + row['key_words_metodo']

def combinar_title_abs(row):
    return row['title_metodo'] + row['abstract_metodo']

def combinar_key_abs(row):
    return row['key_words_metodo'] + row['abstract_metodo']

def combinar_title_abs_key(row):
    return row['title_metodo'] + row['abstract_metodo'] + row['key_words_metodo']

# Combinar palabras en nuevas columnas y eliminar duplicados
combinations = {
    'title_key_metodo': combinar_title_key,
    'title_abs_metodo': combinar_title_abs,
    'key_abs_metodo': combinar_key_abs,
    'title_abs_key_metodo': combinar_title_abs_key
}

for column, comb_function in combinations.items():
    dfA[column] = dfA.apply(comb_function, axis=1)
    dfA[column] = dfA[column].apply(lambda x: list(set(x)))

# Crear listas y diccionarios para mapear campos y nombres
lista_campos = ['key_words_metodo', 'title_metodo', 'abstract_metodo', 'title_key_metodo', 'title_abs_metodo', 'key_abs_metodo', 'title_abs_key_metodo']
nombre_campos = ['PALABRAS CLAVE', 'TÍTULOS', 'RESÚMENES', 'TÍTULO - PALABRAS CLAVE', 'TÍTULO - RESÚMENES', 'PALABRAS CLAVE - RESÚMENES', 'TÍTULO - PALABRAS CLAVE - RESÚMENES']
dic_campo_a_contar_palabras = dict(zip(dic_campo_a_analizar.values(), lista_campos))
dic_nombres_campo = dict(zip(dic_campo_a_analizar.values(), nombre_campos))

# Imprimir mapeo de campos y nombres


## 13 Cálculo de los promedios de los campos analizados

In [14]:
import numpy as np

# Convertir cada lista en su longitud correspondiente
lengths_key = dfA['title_key_metodo'].map(lambda x: len(x))  # Longitudes de la columna 'title_key_metodo'
lengths_title_metodo = dfA['title_metodo'].map(lambda x: len(x))  # Longitudes de la columna 'title_metodo'
lengths_abstract_metodo = dfA['abstract_metodo'].map(lambda x: len(x))  # Longitudes de la columna 'abstract_metodo'
lengths_title_key = dfA['title_key_metodo'].map(lambda x: len(x))  # Longitudes de la columna 'title_key_metodo'
lengths_title_abs_metodo = dfA['title_abs_metodo'].map(lambda x: len(x))  # Longitudes de la columna 'title_abs_metodo'
lengths_key_abs_metodo = dfA['key_abs_metodo'].map(lambda x: len(x))  # Longitudes de la columna 'key_abs_metodo'
lengths_title_abs_key_metodo = dfA['title_abs_key_metodo'].map(lambda x: len(x))  # Longitudes de la columna 'title_abs_key_metodo'

# Calcular el promedio de las longitudes
avg_lengths_key = np.mean(lengths_key)
avg_lengths_title_metodo = np.mean(lengths_title_metodo)
avg_lengths_abstract_metodo = np.mean(lengths_abstract_metodo)
avg_lengths_title_key = np.mean(lengths_title_key)
avg_lengths_title_abs_metodo = np.mean(lengths_title_abs_metodo)
avg_lengths_key_abs_metodo = np.mean(lengths_key_abs_metodo)
avg_lengths_title_abs_key_metodo = np.mean(lengths_title_abs_key_metodo)

# Imprimir promedios de las longitudes
print("El promedio del largo de las listas en la columna 'title_key_metodo' es:", avg_lengths_key)
print("El promedio del largo de las listas en la columna 'title_metodo' es:", avg_lengths_title_metodo)
print("El promedio del largo de las listas en la columna 'abstract_metodo' es:", avg_lengths_abstract_metodo)
print("El promedio del largo de las listas en la columna 'title_key_metodo' es:", avg_lengths_title_key)
print("El promedio del largo de las listas en la columna 'title_abs_metodo' es:", avg_lengths_title_abs_metodo)
print("El promedio del largo de las listas en la columna 'key_abs_metodo' es:", avg_lengths_key_abs_metodo)
print("El promedio del largo de las listas en la columna 'title_abs_key_metodo' es:", avg_lengths_title_abs_key_metodo)


El promedio del largo de las listas en la columna 'title_key_metodo' es: 10.775959123581883
El promedio del largo de las listas en la columna 'title_metodo' es: 7.554256516844202
El promedio del largo de las listas en la columna 'abstract_metodo' es: 58.389581709534944
El promedio del largo de las listas en la columna 'title_key_metodo' es: 10.775959123581883
El promedio del largo de las listas en la columna 'title_abs_metodo' es: 60.85221269593834
El promedio del largo de las listas en la columna 'key_abs_metodo' es: 60.11829912531393
El promedio del largo de las listas en la columna 'title_abs_key_metodo' es: 62.42591149216246


# 14 Definir nombre de campos a analizar

In [15]:
# Diccionario para mapear las variables en función de campo_a_analizar
campo_a_analizar_dict = {
    1: ('key_words_metodo', 'parejas_key_words', 'PALABRAS CLAVE'),
    2: ('title_metodo', 'parejas_title', 'TÍTULOS'),
    3: ('abstract_metodo', 'parejas_abstract', 'RESÚMENES'),
    4: ('title_key_metodo', 'parejas_title_key', 'PALABRAS CLAVE y TÍTULOS'),
    5: ('title_abs_metodo', 'parejas_title_abs', 'TÍTULOS y RESÚMENES'),
    6: ('key_abs_metodo', 'parejas_key_abs', 'PALABRAS CLAVE y RESÚMENES'),
    7: ('title_abs_key_metodo', 'parejas_title_abs_key', 'TÍTULOS, PALABRAS CLAVE y RESÚMENES')
}

# Asignar las variables usando el diccionario y campo_a_analizar
campo_para_nodos_grafo, campo_para_aristas_grafo, nombre_de_campo_analizado = campo_a_analizar_dict.get(campo_a_analizar)

# Imprimir las variables asignadas
print(campo_para_nodos_grafo)
print(campo_para_aristas_grafo)
print(nombre_de_campo_analizado)


title_abs_key_metodo
parejas_title_abs_key
TÍTULOS, PALABRAS CLAVE y RESÚMENES


# 15 Conteo y gráfica del método aplicado al campo elegido

In [16]:
# Obtener el nombre de la columna correspondiente al campo a analizar
campo_a_contar_palabras = dic_campo_a_contar_palabras.get(dic_campo_a_analizar.get(campo_a_analizar))

# Obtener todas las palabras en la columna seleccionada y ordenarlas
campo_con_palabras = sorted(dfA[campo_a_contar_palabras].explode().dropna())

# Contar la frecuencia de cada palabra única en el campo seleccionado
cuenta_campo_a_analizar = dict(Counter(campo_con_palabras))

# Ordenar las palabras por frecuencia de mayor a menor
cuenta_campo_a_analizar_ordenado = sorted(cuenta_campo_a_analizar.items(), key=lambda x: x[1], reverse=True)

# Crear una lista de palabras a eliminar
palabras_a_eliminar = ['', ' ','nan']

# Filtrar las palabras en cuenta_campo_a_analizar_ordenado para excluir palabras en palabras_a_eliminar
cuenta_campo_a_analizar_ordenado_filtrado = [(palabra, frecuencia) for palabra, frecuencia in cuenta_campo_a_analizar_ordenado if palabra not in palabras_a_eliminar]

# Seleccionar las n palabras más frecuentes después de eliminar las palabras no deseadas
cuenta_campo_a_analizar_ordenado_elegidos = cuenta_campo_a_analizar_ordenado_filtrado[:n]

# Calcular porcentajes
porcentajes = [round((frecuencia / len(dfA)) * 100, 1) for palabra, frecuencia in cuenta_campo_a_analizar_ordenado_elegidos]

# Crear texto para cada barra que incluye la frecuencia y el porcentaje
bar_texts = [f"{frecuencia} ({porcentaje}%)" for frecuencia, porcentaje in zip([cuenta_campo_a_analizar_ordenado_elegidos[j][1] for j in range(0, n - 1)], porcentajes)]

#Crear títulos 
title = str(n) + ' palabras clave más frecuentes de ' + nombre_largo


# Crear un gráfico de barras con las palabras más frecuentes después de la eliminación
fig = px.bar(template=template_elegido, orientation='h', y=[cuenta_campo_a_analizar_ordenado_elegidos[j][0] for j in range(0, n - 1)], x=[cuenta_campo_a_analizar_ordenado_elegidos[j][1] for j in range(0, n - 1)], text=bar_texts)

# Actualizar el tamaño de la fuente del texto de las barras
fig.update_traces(textposition='outside', textfont=dict(size=14))  # Ajustar el tamaño y la posición del texto

# Configurar el diseño del gráfico
fig.update_layout(
    title=title,
    yaxis_title="palabras más frecuentes",
    xaxis_title="frecuencia",
    xaxis=dict(range=[0, max([cuenta_campo_a_analizar_ordenado_elegidos[j][1] for j in range(0, n - 1)]) * 1.25]))  # Aumentar el rango del eje x en un 15%)
fig.update_xaxes(tickangle=35)
fig.update_layout(font_size=9)  # Tamaño de la fuente para leyendas y otros elementos

# Mostrar el gráfico
fig.show()
nombre_archivo =abrev +'_'+dic_campo_a_analizar.get(campo_a_analizar)+ "_n_" +str(n)+'_met_'+met+'_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
fig.write_html(abrev + '/' + nombre_archivo+".html")
fig.write_image(abrev + '/' + nombre_archivo+".pdf",width=700, height=800)

# Agregar una sección al archivo LaTeX
entorno_section_tex(ruta_archivo, nombre_archivo_tex, 'Principales palabras de: '+nombre_de_campo_analizado)

# Crear el párrafo para el informe
lista_párrafo4 = ['En la figura \\ref{fig:' + nombre_archivo + '}',
                  'se observan las ', str(n), 'palabras con mayor frecuencia en ', 
                  nombre_de_campo_analizado,' de ',str(len(dfA)),' artículos.',
                  ' Las 3 palabras que más aparecen son ', 
                  '\\textit{', cuenta_campo_a_analizar_ordenado_elegidos[0][0], '},',
                  '\\textit{', cuenta_campo_a_analizar_ordenado_elegidos[1][0], '},',' y ',
                  '\\textit{', cuenta_campo_a_analizar_ordenado_elegidos[2][0], '},',
                  'las cuales, se repiten ', str(cuenta_campo_a_analizar_ordenado_elegidos[0][1]),', ', 
                  str(cuenta_campo_a_analizar_ordenado_elegidos[1][1]),'y ', 
                  str(cuenta_campo_a_analizar_ordenado_elegidos[0][1]),' ', 
                  'veces respectivamente.']

párrafo4 = ' '.join(lista_párrafo4)

print(párrafo4)

# Añadir el párrafo al archivo LaTeX
parrafo_tex(ruta_archivo, nombre_archivo_tex, párrafo4)

# Crear la leyenda para el gráfico
leyenda = 'Principales palabras de: '+nombre_de_campo_analizado+ ' en '+str(len(dfA))+' artículos. Fuente: ' + fuente_articulos + '.'

# Añadir el entorno de figura al archivo LaTeX
entorno_figura_tex(ruta_archivo, nombre_archivo_tex, nombre_archivo, leyenda)
print(leyenda)



# Mostrar las palabras más frecuentes en una lista
[[cuenta_campo_a_analizar_ordenado_elegidos[j][0] for j in range(0, n - 1)]]
print(nombre_archivo)

En la figura \ref{fig:V4_algoritmo_2_title_abs_key_n_35_met_lemmatize_2023_4_27_23_54} se observan las  35 palabras con mayor frecuencia en  TÍTULOS, PALABRAS CLAVE y RESÚMENES  de  23094  artículos.  Las 3 palabras que más aparecen son  \textit{ student }, \textit{ mathematics },  y  \textit{ study }, las cuales, se repiten  12314 ,  12020 y  12314   veces respectivamente.
Principales palabras de: TÍTULOS, PALABRAS CLAVE y RESÚMENES en 23094 artículos. Fuente: Scopus.
V4_algoritmo_2_title_abs_key_n_35_met_lemmatize_2023_4_27_23_54


# 16 Definición de parejas

In [17]:
# Función para crear parejas de palabras únicas en una lista, evitando duplicados y parejas de una palabra consigo misma
def parejas(lista):
    # Crear una lista de tuplas (parejas) con palabras únicas en pares ordenados alfabéticamente y sin repeticiones
    salida1 = [(lista[x], lista[y]) for x in range(len(lista)) for y in range(len(lista)) if y > x]
    return salida1


# Diccionario para asignar un nombre a cada campo a analizar (no se usa en este fragmento de código)
# dic_campo_a_analizar = {1: 'key_words', 2: 'title', 3: 'abstract', 4: 'title_key', 5: 'title_abs', 6: 'key_abs', 7: 'title_abs_key'}

# Crear una columna en el DataFrame dfA con parejas de palabras según el valor de campo_a_analizar
if campo_a_analizar == 1:
    # Crear parejas de palabras clave
    dfA['parejas_key_words'] = dfA['key_words_metodo'].apply(lambda x: sorted(list(set(x)))).transform(parejas)
elif campo_a_analizar == 2:
    # Crear parejas de palabras en el título
    dfA['parejas_title'] = dfA['title_metodo'].apply(lambda x: sorted(list(set(x)))).transform(parejas)
elif campo_a_analizar == 3:
    # Crear parejas de palabras en el resumen
    dfA['parejas_abstract'] = dfA['abstract_metodo'].apply(lambda x: sorted(list(set(x)))).transform(parejas)
elif campo_a_analizar == 4:
    # Crear parejas de palabras en el título y palabras clave
    dfA['parejas_title_key'] = dfA['title_key_metodo'].apply(lambda x: sorted(list(set(x)))).transform(parejas)
elif campo_a_analizar == 5:
    # Crear parejas de palabras en el título y resumen
    dfA['parejas_title_abs'] = dfA['title_abs_metodo'].apply(lambda x: sorted(list(set(x)))).transform(parejas)
elif campo_a_analizar == 6:
    # Crear parejas de palabras en palabras clave y resumen
    dfA['parejas_key_abs'] = dfA['key_abs_metodo'].apply(lambda x: sorted(list(set(x)))).transform(parejas)
else:
    # Crear parejas de palabras en el título, resumen y palabras clave
    dfA['parejas_title_abs_key'] = dfA['title_abs_key_metodo'].apply(lambda x: sorted(list(set(x)))).transform(parejas)


# Las columnas que deseas incluir en dfB
columnas = ['Authors', 'Author(s) ID', 'Title', 'Year',
            'Source title', 'Volume', 'Issue', 'Cited by', 'DOI', 'Link', 'Affiliations',
            'Abstract', 'Author Keywords',
            'Index Keywords', 'Document Type', 'Source', 'EID', 'key_words_non_spot', 'title_non_spot',
            'abstract_non_spot', 'key_words_metodo', 'title_metodo',
            'abstract_metodo', 'title_key_metodo', 'title_abs_metodo',
            'key_abs_metodo', 'title_abs_key_metodo', 'parejas_title_key']

# Crear dfB con solo las columnas que existen en dfA y están en la lista de columnas
dfB = dfA[[col for col in columnas if col in dfA.columns]]

# Exportar dfB a un archivo de Excel
dfB.to_excel(abrev + '/' +"02_df_"+abrev+"_con_parejas.xlsx", index=False)


# 17 Definición de grafo completo

In [18]:
import time
import networkx as nx
from collections import Counter

# Registrar el tiempo de inicio
inicio = time.time()

# Crear un grafo vacío
G = nx.Graph()

# Extraer lista de nodos y aristas de los campos correspondientes en el DataFrame dfA
lista_nodos = [x for row in dfA[campo_para_nodos_grafo] for x in row]
lista_aristas = [x for row in dfA[campo_para_aristas_grafo] for x in row]

# Contar la frecuencia de cada nodo y arista
cuenta_pesos_nodos = Counter(lista_nodos)
cuenta_pesos_aristas = Counter(lista_aristas)

# Calcular los pesos de los nodos y aristas ponderados por la cantidad total de documentos en dfA
cuenta_pesos_nodos_ponderado = {clave: round(valor * 100 / len(dfA), 1) for clave, valor in cuenta_pesos_nodos.items()}
cuenta_pesos_aristas_ponderado = {clave: round(valor * 100 / len(dfA), 1) for clave, valor in cuenta_pesos_aristas.items()}

# Registrar el tiempo de finalización y calcular la duración
fin = time.time()
print(f"{fin-inicio} seg. Equivalentes a {(fin-inicio)/60} min")


from pyvis.network import Network

# Crear nodos en el grafo G con sus atributos correspondientes
for clave, valor in cuenta_pesos_nodos.items():
    ponderado = cuenta_pesos_nodos_ponderado[clave]
    G.add_node(clave, label=clave + ' (' + str(valor) + ')', value=valor, ponderado=ponderado)

# Crear aristas en el grafo G con sus atributos correspondientes
for clave, valor in cuenta_pesos_aristas.items():
    ponderado = cuenta_pesos_aristas_ponderado[clave]
    G.add_edge(clave[0], clave[1], value=valor, label=str(valor), ponderado=ponderado)

# Imprimir información relevante del grafo
print('Información del grafo:')
print('Grafo de la base de datos: ', nombre_largo)
print('Cantidad de artículos: ', len(dfA))
print('Cantidad de nodos grafo completo G: ', len(G.nodes))
print('Cantidad de aristas grafo completo G: ', len(G.edges))
print(dic_nombres_campo.get(dic_campo_a_analizar.get(campo_a_analizar)))

# Registrar el tiempo de finalización y calcular la duración


96.48766303062439 seg. Equivalentes a 1.6081277171770731 min
Información del grafo:
Grafo de la base de datos:  37 revistas de Educación  Matemática 
Cantidad de artículos:  23094
Cantidad de nodos grafo completo G:  51673
Cantidad de aristas grafo completo G:  12527160
TÍTULO - PALABRAS CLAVE - RESÚMENES


In [19]:
nombre_archivo = abrev + '_ERC_G_completo_'+ campo_para_nodos_grafo+ '_met_'+met
nx.write_gml(G, abrev + '/' + nombre_archivo+".gml")
nx.write_graphml(G, abrev + '/' + nombre_archivo+".graphml")


# 18 Información a Latex sobre el Espacio rizomático conceptual (ERC)

# 19 Calcular cantidad de nodos

In [20]:
# Establecer la cantidad de aristas y nodos adecuados para el grafo
cantidad_aristas_adecuadas = 120
cantidad_nodos_adecuados = 120

# Inicializar listas para almacenar la información de los nodos
nodos = []
cantidad_nodos = []

# Iterar a través de un rango de valores (1 a 399)
for i in range(1, 400):
    # Agregar el valor actual de 'i' a la lista de nodos
    nodos.append(i)
    
    # Agregar la cantidad de nodos con un valor mayor que 'i' al grafo G a la lista cantidad_nodos
    cantidad_nodos.append(len([n for n, v in G.nodes(data=True) if v['value'] > i]))

# Crear una lista de pares ordenados con los nodos y sus respectivas cantidades de nodos adecuados
pares_ordenados_nodos = [[nodos[i], cantidad_nodos[i]] for i in range(len(nodos)) if cantidad_nodos[i] <= cantidad_nodos_adecuados]

# Establecer el tamaño del nodo en función del primer par ordenado en la lista
tamaño_nodo = pares_ordenados_nodos[0][0]

# Imprimir el tamaño del nodo, la cantidad de nodos y los pares ordenados de nodos
print(tamaño_nodo)

import plotly.express as px
title='Cantidad de nodos según el peso mínimo del nodo'
fig = px.scatter(x=nodos, y=cantidad_nodos, title=title)
fig.update_layout(
    title=title,
    xaxis_title="peso mínimo de los nodos",
    yaxis_title="cantidad de nodos")
fig.show()


#title='Cantidad de nodos según el peso mínimo del nodo (zoom)'
#fig = px.scatter(x=nodos, y=cantidad_nodos, title=title)
#fig.update_layout(
#    title=title,
#    xaxis_title="peso mínimo de los nodos",
#    yaxis_title="cantidad de nodos")
#fig.update_yaxes(range=[0, 200])
#fig.show() 

# Contar nodos de tamaño 1 y 2
nodos_tamaño_1 = len([n for n, v in G.nodes(data=True) if v['value'] == 1])
nodos_tamaño_2 = len([n for n, v in G.nodes(data=True) if v['value'] == 2])

# Imprimir la cantidad de nodos de tamaño 1 y 2
print("Cantidad de nodos de tamaño 1:", nodos_tamaño_1)
print("Cantidad de nodos de tamaño 2:", nodos_tamaño_2)


# Imprimir la leyenda y el tamaño del nodo

print(tamaño_nodo)


# Contar aristas de tamaño 1 y 2
aristas_tamaño_1 = len([(u, v) for u, v, d in G.edges(data=True) if d['value'] == 1])
aristas_tamaño_2 = len([(u, v) for u, v, d in G.edges(data=True) if d['value'] == 2])

# Imprimir la cantidad de aristas de tamaño 1 y 2
print("Cantidad de aristas de tamaño 1:", aristas_tamaño_1)
print("Cantidad de aristas de tamaño 2:", aristas_tamaño_2)



# Importar la biblioteca plotly para visualización de datos
import plotly.express as px


# Crear listas vacías para almacenar aristas y su cantidad
aristas = []
cantidad_aristas = []

# Calcular la cantidad de aristas para cada umbral de 1 a 400
for j in range(1, 400):
    aristas.append(j)
    cantidad_aristas.append(len(dict(filter(lambda x: x[1] >= j, cuenta_pesos_aristas.items()))))


# Crear una lista de pares ordenados para las aristas que cumplan con el criterio de cantidad adecuada
pares_ordenados_aristas = [[aristas[i], cantidad_aristas[i]] for i in range(0, len(aristas)) if cantidad_aristas[i] <= cantidad_aristas_adecuadas]

# Obtener el valor del filtro para las aristas
filtro_arista = pares_ordenados_aristas[0][0]

# Imprimir el valor del filtro y la lista de pares ordenados
print(filtro_arista)
#print(pares_ordenados_aristas)


title='Cantidad de aristas  según peso mínimo de las aristas'

fig = px.scatter(x=aristas, y=cantidad_aristas, title=title)
fig.update_layout(
    title=title,
    xaxis_title="peso mínimo de las aristas",
    yaxis_title="cantidad de aristas")

fig.show()





# Imprimir el tamaño del nodo
print(tamaño_nodo)





IndexError: list index out of range

# 20 Definición manual del tamaño de nodos y aristas

In [ ]:
#Cambio de cantidad de nodos y aristas manual
#hay que borrar los símbolos # que están delante de las dos líneas siguientes

#tamaño_nodo = 2
#filtro_arista = 3


# 21 Grafo completo con nodos más frecuentes remarcados

In [ ]:
# Ordenar cuenta_pesos_nodos en orden descendente
cuenta_pesos_nodos_ordenado = dict(sorted(cuenta_pesos_nodos.items(), key=lambda item: item[1], reverse=True))

# Obtener los j nodos más frecuentes
j = 10
lista_nodos_frecuentes = list(cuenta_pesos_nodos_ordenado.keys())[:j]


# Filtrar aristas según el umbral 'filtro_arista'
cuenta_pesos_aristas_H = dict(filter(lambda x: x[1] >= filtro_arista, cuenta_pesos_aristas.items()))

# Crear un grafo vacío
H = nx.Graph()


# Crear y agregar nodos al grafo H según el modo de visualización (Absoluto o Ponderado)
if modo_grafo == 1:  # Modo absoluto
    for clave, valor in cuenta_pesos_nodos.items():
        if clave in lista_nodos_frecuentes:
            H.add_node(clave, label=f"{clave} ({valor})", value=valor, color='purple')
        else: 
            H.add_node(clave, label=f"{clave} ({valor})", value=valor, color='#5a7497')
else:  # Modo ponderado
    for clave, valor in cuenta_pesos_nodos.items():
        if clave in lista_nodos_frecuentes:
            H.add_node(clave, label=f"{clave} ({cuenta_pesos_nodos_ponderado.get(clave)}%)", value=cuenta_pesos_nodos_ponderado.get(clave), color='purple')
        else: 
            H.add_node(clave, label=f"{clave} ({cuenta_pesos_nodos_ponderado.get(clave)}%)", value=cuenta_pesos_nodos_ponderado.get(clave), color='#5a7497')
        
    # Agregar aristas al grafo H
    for clave, valor in cuenta_pesos_aristas_H.items():
        H.add_edge(clave[0], clave[1], value=cuenta_pesos_aristas_ponderado.get(clave), label=f"{cuenta_pesos_aristas_ponderado.get(clave)}%", color='#97c2fc')

# Filtrar nodos del grafo H según el tamaño mínimo del nodo
selected_nodes = [n for n, v in G.nodes(data=True) if v['value'] >= tamaño_nodo]
H = H.subgraph(selected_nodes)

# Mostrar información sobre el grafo
print(nombre_largo)
print(f"Cantidad de nodos subgrafo completo H: {len(H.nodes)}")
print(f"Cantidad de aristas subgrafo completo H: {len(H.edges)}")
print(f"Nodos de tamaño como mínimo: {tamaño_nodo}")
print(f"Aristas de tamaño como mínimo: {filtro_arista}")

# Visualizar el grafo usando la biblioteca pyvis
net = Network(notebook=False, heading=f"Espacio Rizomático Conceptual de {nombre_largo}, n={len(dfA)} artículos")
net.repulsion()
net.from_nx(H)

# Guardar y mostrar el grafo en un archivo HTML
nombre_archivo = f"grafo_{abrev}{dic_modo_grafo.get(modo_grafo)}_{campo_para_nodos_grafo}_met_{met}_n_{len(dfA.index)}__f_nodo_{tamaño_nodo}_f_edge_{filtro_arista}"
net.show(f"{abrev}/{nombre_archivo}.html")

# Imprimir la leyenda del gráfico para ser utilizada en un documento LaTeX
print(f"\\caption{{Extracto del Espacio Rizomático Conceptual para {nombre_largo} que incluye {len(dfA)} artículos. En este grafo se muestran solo los nodos de cuyo tamaño es al menos {tamaño_nodo} y las aristas de al menos tamaño {filtro_arista}.}}")
print(nombre_archivo)
        

# 22 exportar a Datapane

# 23 Diccionario con palabras en el grafo G

In [ ]:
lista_nodosG = [e for e in G.nodes(data=True)]
lista_nodosG_ordenada = sorted(lista_nodosG, key=lambda x: x[1]['value'], reverse=True)  # [:20]

parte_de_palabra = 'spa'
nodos_con_parte_de_palabra = [nodo for nodo in lista_nodosG_ordenada if nodo[0].startswith(parte_de_palabra)]


if not nodos_con_parte_de_palabra:
    print(f"No se encontraron nodos que contengan '{parte_de_palabra}'")
else:
    nodos_con_letra = nodos_con_parte_de_palabra
    print('Palabras que comiencen con la cadena :',parte_de_palabra)
    for x in range(0, len(nodos_con_letra)):
        print(nodos_con_letra[x][1].get('label'))


# 24 Finalizar informe LaTex

In [ ]:
import plotly.express as px



# Obtener las aristas con sus valores y ordenarlas de acuerdo con su valor
aristas_valores = [(u, v, d['value']) for u, v, d in G.edges(data=True)]
aristas_valores.sort(key=lambda x: x[2], reverse=True)

# Seleccionar las n aristas con valores más altos
aristas_top_n = aristas_valores[:n]

# Crear etiquetas para las aristas
etiquetas_aristas = [f"{u}-{v}" for u, v, _ in aristas_top_n]

# Extraer los valores de las aristas
valores_aristas = [valor for _, _, valor in aristas_top_n]



# Calcular los porcentajes de cada arista en relación a len(dfA)
porcentajes_aristas = [(valor / len(dfA)) * 100 for valor in valores_aristas]

# Crear texto con valores y porcentajes para cada barra
bar_texts = [f"{valor} ({porcentaje:.1f}%)" for valor, porcentaje in zip(valores_aristas, porcentajes_aristas)]

# Crear título
title = f"{n} aristas más frecuentes en {nombre_largo}"

# Crear un gráfico de barras con Plotly Express
fig = px.bar(y=etiquetas_aristas, x=valores_aristas, 
             text=bar_texts, orientation='h',
             template=template_elegido)

# Configurar el diseño del gráfico
fig.update_layout(
    title=title,
    yaxis_title="Aristas",
    xaxis_title="Frecuencia",
    xaxis=dict(range=[0, max(valores_aristas) * 1.25])
)
fig.update_xaxes(tickangle=35)
fig.update_layout(font_size=9)

# Actualizar el tamaño de la fuente del texto de las barras
fig.update_traces(textposition='outside', textfont=dict(size=14))

# Mostrar el gráfico
fig.show()
nombre_archivo =abrev +'_aristas_'+dic_campo_a_analizar.get(campo_a_analizar)+ "_n_" +str(n)+'_met_'+met+'_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
fig.write_html(abrev + '/' + nombre_archivo+".html")
fig.write_image(abrev + '/' + nombre_archivo+".pdf",width=700, height=800)

# Crear la leyenda para el gráfico
leyenda = 'Principales aristas de: '+nombre_de_campo_analizado+ ' en '+str(len(dfA))+' artículos. Fuente: ' + fuente_articulos + '.'

# Añadir el entorno de figura al archivo LaTeX
entorno_figura_tex(ruta_archivo, nombre_archivo_tex, nombre_archivo, leyenda)
print(leyenda)


# Agregar una nueva página al archivo LaTeX
entorno_nueva_pagina_tex(ruta_archivo, nombre_archivo_tex)

# Agregar una sección al archivo LaTeX con el título especificado
entorno_section_tex(ruta_archivo, nombre_archivo_tex, 'Espacio rizomático conceptual (\\textit{ERC}) a partir de las palabras en: ' + dic_nombres_campo.get(dic_campo_a_analizar.get(campo_a_analizar)))

# Agregar una subsección al archivo LaTeX con el título especificado
entorno_sub_section_tex(ruta_archivo, nombre_archivo_tex, 'Información general del ERC')

# Agregar un párrafo al archivo LaTeX
parrafo_tex(ruta_archivo, nombre_archivo_tex, 'La información general del ERC es la siguiente:')

# Crear una lista con la información general del ERC
lista_información_general = [
    'Grafo de la base de datos denominada: ' + nombre_largo,
    'El método utilizado fue: ' + met,
    'Cantidad de artículos: ' + str(len(dfA)),
    'Cantidad de nodos grafo completo G: ' + str(len(G.nodes)),
    'Cantidad de aristas grafo completo G: ' + str(len(G.edges)),
    'Tiempo requerido para generar el grafo: ' + str(round(fin - inicio, 1)) + ' seg. Equivalentes a ' + str(round((fin - inicio) / 60, 1)) + ' min.'
]

# Agregar la lista de información general del ERC al archivo LaTeX usando el entorno "itemize"
entorno_itemize_tex(ruta_archivo, nombre_archivo_tex, lista_información_general)

# Imprimir la lista de información general del ERC
print(lista_información_general)

# Imprimir el título de la sección del ERC
print('Espacio rizomático conceptual (ERC) a partir de las palabras en: ' + dic_nombres_campo.get(dic_campo_a_analizar.get(campo_a_analizar)))

# Agregar una subsección en el archivo LaTeX con el título "Nodos"
entorno_sub_section_tex(ruta_archivo, nombre_archivo_tex, 'Nodos')

# Crear una lista con información sobre los nodos del grafo
lista_informacion_nodos = [
    'Como se indicó en la información general, el \\textit{ERC} tiene ' + str(len(G.nodes)) + ' nodos.',
    'Para que en el \\textit{ERC} se puedan distinguir los nodos más importantes,',
    ' se estableció como una cantidad adecuada para visualizar el ERC, ' + str(cantidad_nodos_adecuados),
    ' nodos. Esto da como resultado, que el \\textit{ERC} mostrará los nodos que tengan un tamaño igual o superior a ',
    str(tamaño_nodo) + '. Es decir, si una palabra se repite al menos ',
    str(tamaño_nodo) + ' veces en los ' + str(len(dfA)),
    ' documentos analizados, entonces aparecerá en el Espacio Rizomático Conceptual de ',
    nombre_largo + '. Se debe notar que esto, en términos porcentuales ',
    'significa que si una palabra se repite al menos un ',
    str(round(100*tamaño_nodo/len(dfA),1))+'\\% entonces aparecerá en el \\textit{ERC}. \\newline',
    'Entre los nodos que se descartan, están aquellos que aparecen pocas veces y que son numerosos. ',
    'Por ejemplo, hay ', str(nodos_tamaño_1), ' palabras que solo aparecen una vez y ',
    str(nodos_tamaño_2), ' palabras que solo aparecen dos veces. Estos nodos y todos aquellos que quedan ',
    'bajo el umbral quedan en la \\textbf{periferia invisible}.'
]

# Unir la lista de información de nodos en un solo párrafo
parrafo = ' '.join(lista_informacion_nodos)

# Imprimir y agregar el párrafo al archivo LaTeX
print(parrafo)
parrafo_tex(ruta_archivo, nombre_archivo_tex, parrafo)


# Agregar una subsección en el archivo LaTeX con el título "Aristas"
entorno_sub_section_tex(ruta_archivo, nombre_archivo_tex, 'Aristas')

# Crear una lista con información sobre los nodos del grafo
lista_informacion_nodos = [
    'Como se indicó en la información general, el \\textit{ERC} tiene ' + str(len(G.edges)),
    ' aristas. Para que en el \\textit{ERC} se puedan distinguir las aristas más importantes,',
    ' se estableció como una cantidad adecuada para visualizar el \\textit{ERC}, ',
    str(cantidad_nodos_adecuados) + ' aristas. Esto da como resultado,', 
    'que el \\textit{ERC} mostrará las aristas que tengan un tamaño igual o superior a ',
    str(filtro_arista) + '. Es decir, si una conexión entre dos palabras se repite',
    ' al menos ' + str(filtro_arista) + ' veces en los ' + str(len(dfA)),
    ' documentos analizados, entonces aparecerá en el \\textit{ERC} de ' + nombre_largo + '. \\newline',
    'en términos porcentuales significa que si una palabra se repite al menos un ',
    str(round(100*filtro_arista/len(dfA),1))+'\\% entonces aparecerá en el \\textit{ERC}. \\newline',
    'Entre las aristas que se descartan, están aquellas que aparecen pocas veces y que son numerosos. ',
    'Por ejemplo, hay ', str(aristas_tamaño_1), ' parejas de palabras que solo aparecen una vez y ',
    str(aristas_tamaño_2), ' parejas de palabras que solo aparecen dos veces. Estas aristas y todos las que quedan ',
    'bajo el umbral son parte de la \\textbf{periferia invisible} del \\textit{ERC}.'
]

# Unir la lista de información de nodos en un solo párrafo
parrafo = ' '.join(lista_informacion_nodos)

# Imprimir y agregar el párrafo al archivo LaTeX
print(parrafo)
parrafo_tex(ruta_archivo, nombre_archivo_tex, parrafo)

lista_información_grafo=['Por lo tanto ',
r"$\mathcal{G}_{\eta=",
str(tamaño_nodo),
r",\alpha=",str(filtro_arista),
r"}=(V_{\eta=",str(tamaño_nodo),
r",\alpha=",str(filtro_arista),r"},E_{\eta=",  
str(tamaño_nodo),r",\alpha=",str(filtro_arista),'})$',  
'es un grafo tal que, los nodos tienen tamaño mayor o igual a ', 
str(tamaño_nodo),' y las aristas tienen tamaño mayor o igual a ',
str(filtro_arista)+'.',' Esto produce un grafo que tiene ',str(len(H.nodes)),
'nodos y ', str(len(H.edges)), 'aristas. ','\n',
'En la figura \\ref{fig:'+nombre_archivo+'} se muestra un gráfico con los ' + str(n) +' nodos con mayor frecuencia.'                         
                         ]

párrafo=' '.join(lista_información_grafo)
parrafo_tex(ruta_archivo, nombre_archivo_tex, párrafo)

parrafo_tex(ruta_archivo, nombre_archivo_tex, '\\bibliographystyle{apalike-es.bst}')
parrafo_tex(ruta_archivo, nombre_archivo_tex, '\\bibliography{referencias.bib}')
parrafo_tex(ruta_archivo, nombre_archivo_tex, '\\end{document}')


# 25 Podar el rizoma

In [ ]:

#actualizado abril 27 16:57
lista_nodos= [e for e in G.nodes(data=True)][:10]
#print(lista_nodos)
tamaño_nodo = int(len(dfA)*0.01)
filtro_arista = int(len(dfA)*0.02)
cuenta_pesos_aristas_H1=dict(filter(lambda x: x[1] >= filtro_arista, cuenta_pesos_aristas.items()))

lista_palabras_ed_mat = ["anthropological","apos","argumentation","pck", "assessment","belief", "cognitive", "communication", "competence", "concept", 
                         "conceptual", "conception", "content", "curriculum", "didactic", "digital", "discourse", "dynamic","ethnomathematics"
                         , "early", "game", "history", "inquiry", "inquirybased", "knowledge", "literacy","meaning", "modeling", "modelling"
                         , "noticing", "online", "ontosemiotic" , "pedagogical", "problem", "problemsolving", "reform",
                         "representation", "resource", "semiotic", "skill" , "solving", "stem", "strategy", "task", "technology", 
                         "textbook", "tool", "video", "visualization",'perception','assessing', 'attitude', 'computer', 'error', 'ict',
                         'misconception', 'motivation', 'anthropological', 'assessing', 'attitude', 'cultural', 'computational', 'computer', 
                         'decomposition', 'didactical', 'didactics', 'epistemological', 'epistemology', 'error', 
                         'evaluation', 'genetic', 'geogebra', 'historical', 'ict', 'misconception', 'motivation', 
                         'perception', 'philosophy', 'sense', 'socioepistemology', 'software', 'visual']
lista_palabras_matemáticas = ["numerical",'proof',"proportionality",'programming',"addition", "algebra", "algebraic", "arithmetic","biology", "calculus", "change","chemistry", "complex", "decimal", "definition", "differential",'derivative','engineering', "equation", "fibonacci", "fraction", "function", "geometric", "geometry", "graph", "integral", "integer", "line", "linear", "mathematical", "mathematics","math", "maths", "matrix", "measurement",'multiplication', "negative", "number", "operation", "pattern","physic","physical","physics" ,"probability", "rational", "science",'scientific', "sequence",'simulation' ,"space", "spatial", "statistic", "statistical", "theorem", "triangle", "variable", "whole"]


lista_palabras_enseñanza = ["development","initial","instruction", "instructional","inservice","pedagogical" , "practice", "preservice", "prospective","professional", "teacher", "teaching", "training"]
lista_palabras_enseñanza_aprendizaje2 = ["education","elementary",'middle', 'college', 'pedagogy',"class", "classroom", "high","lesson", "primary", "school","secondary", "university", "undergraduate"]
lista_palabras_aprendizaje = ["learn","learning", "learner", "reading", "reasoning", "reflection", "student", "thinking", "understanding"]
social = ['mexican','social','brazil', 'integration','community', 'equity', 'collaborative', 'opportunity','africa', 'african', 'botswana', 'critical',"collaboration","competency", 'culture', 'gender', 'identity', 'indigenous', 'kenya', 'lesotho', 'malawi', 'multilingual', 'nigeria', 'policy', 'south', 'zimbabwe']
otras_palabras=['academic', 'achievement', 'activity', 'africa', 'african', 'analysis', 'approach', 'attitude', 'case', 'classroom', 'community', 'computer', 'context', 'course', 'design', 'developing', 'effect', 'engineering', 'experience', 'exploring', 'factor', 'framework', 'gender', 'grade', 'high', 'language', 'level', 'method', 'model', 'perception', 'performance', 'perspective', 'primary', 'relationship', 'school', 'scientific', 'secondary', 'south', 'system', 'undergraduate', 'university', 'using', 'view']
lista_palabras = ['area', 'art', 'based', 'basic', 'child', 'connection', 'construction', 'contribution', 'data', 'degree', 'difficulty', 'different', 'distance', 'educational', 'educator', 'environment', 'experiment', 'field', 'first', 'future', 'group', 'image', 'information', 'interaction', 'interpretation', 'issue', 'material', 'methodology', 'modern', 'narrative', 'national', 'new', 'notion', 'possibility', 'process', 'production', 'program', 'proposal', 'real', 'relation', 'review', 'role', 'situation', 'state', 'structure', 'suitability', 'test', 'theoretical', 'theory', 'towards', 'transformation', 'work', 'working', 'world', 'year']
otras_palabras=otras_palabras+lista_palabras

lista_de_palabras_con_color = lista_palabras_ed_mat+lista_palabras_matemáticas+lista_palabras_enseñanza+lista_palabras_enseñanza_aprendizaje2+lista_palabras_aprendizaje+social+otras_palabras

lista_nodos_a_colorear_1 = lista_palabras_ed_mat
lista_nodos_a_colorear_2 = lista_palabras_matemáticas
lista_nodos_a_colorear_3a = lista_palabras_enseñanza + lista_palabras_aprendizaje
lista_nodos_a_colorear_3b = lista_palabras_enseñanza_aprendizaje2
lista_nodos_a_colorear_4 = social #lista_palabras_aprendizaje

H1 = nx.Graph()
#creación de nodos de H1

if modo_grafo == 1:
    #for clave, valor in nodos_color_1.items():
    #    H1.add_node(clave,label=clave+' ('+str(valor)+')', value=(valor) , color = '#eaf2fe') #5a7497
    #for clave, valor  in nodos_color_2.items():
    #    H1.add_node(clave,label=clave+' ('+str(valor)+')', value=(valor) , color = '#eaf2fe') #97c2fc
        
    for x in lista_nodos_a_colorear_1:
        H1.add_node(x,label=x+' ('+str(cuenta_pesos_nodos.get(x))+')'
               ,value=(cuenta_pesos_nodos.get(x))
               ,color = "purple")
    for x in lista_nodos_a_colorear_2:
        H1.add_node(x,label=x+' ('+str(cuenta_pesos_nodos.get(x))+')'
               ,value=(cuenta_pesos_nodos.get(x))
               ,color = "orange")
    for x in lista_nodos_a_colorear_3a:
        H1.add_node(x,label=x+' ('+str(cuenta_pesos_nodos.get(x))+')'
               ,value=(cuenta_pesos_nodos.get(x))
               ,color = "green")
    for x in lista_nodos_a_colorear_3b:
        H1.add_node(x,label=x+' ('+str(cuenta_pesos_nodos.get(x))+')'
               ,value=(cuenta_pesos_nodos.get(x))
               ,color = "#93bf85")
    for x in lista_nodos_a_colorear_4:
        H1.add_node(x,label=x+' ('+str(cuenta_pesos_nodos.get(x))+')'
               ,value=(cuenta_pesos_nodos.get(x))
               ,color = "#00cdff")
    for clave, valor in cuenta_pesos_aristas_H1.items():
        H1.add_edge(clave[0],clave[1], value = (valor),label=str(valor),color = '#97c2fc') 



else:
    #for clave, valor in nodos_color_1.items():
    #    H1.add_node(clave,label=clave+' ('+str(cuenta_pesos_nodos_ponderado.get(clave))+'%)', value=(cuenta_pesos_nodos_ponderado.get(clave)) , color = '#eaf2fe') #5a7497
    
    #for clave, valor in nodos_color_2.items():
    #    H1.add_node(clave,label=clave+' ('+str(cuenta_pesos_nodos_ponderado.get(clave))+'%)', value=(cuenta_pesos_nodos_ponderado.get(clave)) , color = '#eaf2fe')
    
    for x in lista_nodos_a_colorear_1:
        H1.add_node(x,label=x+' ('+str(cuenta_pesos_nodos_ponderado.get(x))+'%)'
               ,value=(cuenta_pesos_nodos_ponderado.get(x))
               ,color = "purple")
    for x in lista_nodos_a_colorear_2:
        H1.add_node(x,label=x+' ('+str(cuenta_pesos_nodos_ponderado.get(x))+'%)'
               ,value=(cuenta_pesos_nodos_ponderado.get(x))
               ,color = "orange")
    for x in lista_nodos_a_colorear_3a:
        H1.add_node(x,label=x+' ('+str(cuenta_pesos_nodos_ponderado.get(x))+'%)'
               ,value=(cuenta_pesos_nodos_ponderado.get(x))
               ,color = "green")
    for x in lista_nodos_a_colorear_3b:
        H1.add_node(x,label=x+' ('+str(cuenta_pesos_nodos_ponderado.get(x))+'%)'
               ,value=(cuenta_pesos_nodos_ponderado.get(x))
               ,color = "#93bf85")
    for x in lista_nodos_a_colorear_4:
        H1.add_node(x,label=x+' ('+str(cuenta_pesos_nodos_ponderado.get(x))+'%)'
               ,value=(cuenta_pesos_nodos_ponderado.get(x))
               ,color = "#00cdff")
    for clave, valor in cuenta_pesos_aristas_H1.items():
        H1.add_edge(clave[0],clave[1], value = (cuenta_pesos_aristas_ponderado.get(clave)),label=str(cuenta_pesos_aristas_ponderado.get(clave))+'%',color = '#97c2fc') 


palabras_a_podar=['nan',' ','','']

# lista_de_palabras_con_color+
#palabras_a_podar = ['nan','study','research',' ','','','mathematics','education','mathematical']
#palabras_a_podar=['nan','mathematical','education','mathematics','study']

selected_nodes = [n for n,v in G.nodes(data=True) if v['value'] >= tamaño_nodo and n not in palabras_a_podar]
H1 = H1.subgraph(selected_nodes)

fin = time.time()
#net=Network(notebook=False, heading='Espacio Rizomático Conceptual PODADO de '+ nombre_largo +', n='+str(len(dfA))+' artículos')
net=Network(notebook=False)

net.repulsion()
net.from_nx(H1)
nombre_archivo = 'grafo_podado_'+ abrev +dic_modo_grafo.get(modo_grafo)+  campo_para_nodos_grafo + '_met_'+ met +str(len(dfA)) +'_n_' + str(len(dfA)) + '_f_nodo_'+ str(tamaño_nodo) + '_f_edge_'+ str(filtro_arista)
net.show(abrev + '/' + nombre_archivo+'.html')


print('----------------------------')
lista_nodosH1= [e for e in H1.nodes(data=True)]
lista_nodosH1_ordenada = sorted([e for e in lista_nodosH1 if 'value' in e[1]], key=lambda x: x[1]['value'], reverse=True)

#lista_nodosH1_ordenada = sorted(lista_nodosH1, key=lambda x: x[1]['value'], reverse=True) # [:20]


Afiliación='global'
caption_1= '\caption{'+'Extracto del '+ '$\mathcal{G}_{'+str(tamaño_nodo)+','+str(filtro_arista)+'}$ Espacio Rizomático Conceptual'+' \\textbf{podado} para ' + str(len(dfA)) +  ' artículos con afiliación en países de \\textbf{'+ Afiliación  +'}. Las palabras podadas son: \\textit{education}, \\textit{mathematics} y \\textit{mathematical}. '
código_color=' Código de colores: \\textcolor[HTML]{800080}{$\\bullet$} conceptos de educación matemática; \\textcolor[HTML]{ffa500}{$\\bullet$} conceptos matemáticos disciplinares; \\textcolor[HTML]{008000}{$\\bullet$} y \\textcolor[HTML]{93bf85}{$\\bullet$}  conceptos genéricos de educación; \\textcolor[HTML]{00cdff}{$\\bullet$} conceptos sobre temas sociales  y \\textcolor[HTML]{eaf2fe}{$\\bullet$} conceptos que no están en las otras categorías.}' 
caption_1=caption_1+código_color
print(caption_1)

print('filtro arista:', filtro_arista)
print('tamaño_nodo: ',tamaño_nodo)

print('\n')
print('Lista de aristas y pesos de ERC podado: ',)
#for x in lista_nodosH1_ordenada:
#    print(x[0],':',x[1].get('value'))

lista_aristas_H1_ordenada=[e for e in H1.edges(data=True)]
lista_aristas_H1_ordenada = sorted(lista_aristas_H1_ordenada, key=lambda x: x[2]['value'], reverse=True)



#for x in lista_aristas_H1_ordenada:
    #print(x[0],'-',x[1],':',x[2].get('value'))



# 26 Exportar a Datapane el grafo podado


import datapane as dp

with open(abrev + '/'+nombre_archivo+'.html', 'r') as f:
    html_network_1 = f.read()
    
dp.Report(
  dp.HTML(
    html_network_1
  )
).upload(name=abrev+' - espacio rizomático conceptual PODADO '+ str(len(dfA)) + ' artículos')

# 27 Ego de un grafo

In [ ]:
ego='argumentationeeee'
lista_nodos_a_colorear =[ego]
tamaño_nodo = 2
filtro_arista = 6

if modo_grafo == 1:
    tamaño_nodo = tamaño_nodo
    tamaño_nodo_abs = tamaño_nodo
    print('tamaño_nodo_abs:',tamaño_nodo_abs)
    filtro_arista = filtro_arista
    filtro_arista_abs = filtro_arista
else:
    tamaño_nodo_abs = tamaño_nodo
    tamaño_nodo = tamaño_nodo/len(dfA)
    print('tamaño_nodo_abs:',tamaño_nodo_abs)
    filtro_arista_abs = filtro_arista
    filtro_arista = filtro_arista/len(dfA)
    

cuenta_pesos_aristas_Ego=dict(filter(lambda x: x[1] >= filtro_arista_abs, cuenta_pesos_aristas.items()))

Ego = nx.Graph()




if modo_grafo == 1:
    #for clave, valor in nodos_color_1.items():
    #    Ego.add_node(clave,label=clave+' ('+str(valor)+')', value=(valor) , color = '#5a7497')
    #for clave, valor  in nodos_color_2.items():
    #    Ego.add_node(clave,label=clave+' ('+str(valor)+')', value=(valor) , color = '#97c2fc')
        
    for x in lista_nodos_a_colorear_1:
        Ego.add_node(x,label=x+' ('+str(cuenta_pesos_nodos.get(x))+')'
               ,value=(cuenta_pesos_nodos.get(x))
               ,color = "purple")
    for x in lista_nodos_a_colorear_2:
        Ego.add_node(x,label=x+' ('+str(cuenta_pesos_nodos.get(x))+')'
               ,value=(cuenta_pesos_nodos.get(x))
               ,color = "green")
    for x in lista_nodos_a_colorear_3a:
        Ego.add_node(x,label=x+' ('+str(cuenta_pesos_nodos.get(x))+')'
               ,value=(cuenta_pesos_nodos.get(x))
               ,color = "pink")
    for x in lista_nodos_a_colorear_3b:
        Ego.add_node(x,label=x+' ('+str(cuenta_pesos_nodos.get(x))+')'
               ,value=(cuenta_pesos_nodos.get(x))
               ,color = "orange")
    for clave, valor in cuenta_pesos_aristas_Ego.items():
        Ego.add_edge(clave[0],clave[1], value = (valor),label=str(valor),color = '#97c2fc') 



else:
    #for clave, valor in nodos_color_1.items():
    #    Ego.add_node(clave,label=clave+' ('+str(cuenta_pesos_nodos_ponderado.get(clave))+'%)', value=(cuenta_pesos_nodos_ponderado.get(clave)) , color = '#5a7497')
    
    #for clave, valor in nodos_color_2.items():
    #    Ego.add_node(clave,label=clave+' ('+str(cuenta_pesos_nodos_ponderado.get(clave))+'%)', value=(cuenta_pesos_nodos_ponderado.get(clave)) , color = '#97c2fc')
    
    for x in lista_nodos_a_colorear_1:
        Ego.add_node(x,label=x+' ('+str(cuenta_pesos_nodos_ponderado.get(x))+'%)'
               ,value=(cuenta_pesos_nodos_ponderado.get(x))
               ,color = "purple")
    for x in lista_nodos_a_colorear_2:
        Ego.add_node(x,label=x+' ('+str(cuenta_pesos_nodos_ponderado.get(x))+'%)'
               ,value=(cuenta_pesos_nodos_ponderado.get(x))
               ,color = "green")
    for x in lista_nodos_a_colorear_3a:
        Ego.add_node(x,label=x+' ('+str(cuenta_pesos_nodos_ponderado.get(x))+'%)'
               ,value=(cuenta_pesos_nodos_ponderado.get(x))
               ,color = "pink")
    for x in lista_nodos_a_colorear_3b:
        Ego.add_node(x,label=x+' ('+str(cuenta_pesos_nodos_ponderado.get(x))+'%)'
               ,value=(cuenta_pesos_nodos_ponderado.get(x))
               ,color = "orange")
    for clave, valor in cuenta_pesos_aristas_Ego.items():
        Ego.add_edge(clave[0],clave[1], value = (cuenta_pesos_aristas_ponderado.get(clave)),label=str(cuenta_pesos_aristas_ponderado.get(clave))+'%',color = '#97c2fc') 

palabras_a_podar=['nan','study','research',' ','','','mathematics','education','mathematical']
#palabras_a_podar=['nan','mathematical','education','mathematics','study']





#palabras_a_podar=['nan','mathematics','education','mathematical']

selected_nodes = [n for n,v in G.nodes(data=True) if v['value'] >= tamaño_nodo_abs and n not in palabras_a_podar]

    
if ego in G:
    Ego = Ego.subgraph(selected_nodes)
    
    hub_ego = nx.ego_graph(Ego, ego, radius=1)

    for u, v in list(hub_ego.edges):
        if u != ego and v != ego:
            hub_ego.remove_edge(u, v)
    
    net = Network(notebook=True)
    net = Network(notebook=False, heading='EGO Rizomático Conceptual de '+ ego +', basado en '+ nombre_largo +', n='+str(len(dfA))+' artículos')
    net.repulsion()
    net.from_nx(hub_ego)
    nombre_archivo = 'grafo_ego_'+ego+'_'+ abrev +dic_modo_grafo.get(modo_grafo) + campo_para_nodos_grafo + '_met_'+ met +str(len(dfA)) +'_n_' + str(len(dfA.index)) + '_f_nodo_'+ str(tamaño_nodo_abs) + '_f_edge_'+ str(filtro_arista_abs)
    net.show(abrev + '/'+nombre_archivo+'.html')
    lista_aristas_hub_ego = [e for e in hub_ego.edges(data=True)]
    lista_aristas_hub_ego = sorted(lista_aristas_hub_ego, key=lambda x: x[2]['value'], reverse=True)
    
    print('\caption{','Ego del nodo',ego, ' para ',nombre_largo, 'que incluye' ,len(dfA), 
          ' artículos. En este grafo se muestran solo los nodos de cuyo tamaño es al menos ',str(tamaño_nodo),
          ' y las aristas de al menos tamaño ',filtro_arista,'.}')
    
    print('Grafo de la base: ',nombre_largo)
    print('Cantidad de artículos: ',len(dfA))
    print('Cantidad de nodos grafo completo G: ',(len(hub_ego.nodes)))
    print('Cantidad de aristas grafo completo G: ',(len(hub_ego.edges)))
    
    print('\n')
    print('Lista de aristas y pesos del EGO: ',ego)
    for x in lista_aristas_hub_ego[30:]:
        print(x[0],'-',x[1],':',x[2].get('value'))

else:
    print("El ego: <<",ego, ">> no está presente en el grafo G.")


In [ ]:
fin = time.time()
print(f"{fin-inicio} seg. Equivalentes a {(fin-inicio)/60} min")



# 28 Exportar a Datapane

# 29 Información para crear